In [1]:
!pip install faker psycopg2


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\asusz\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Obtaining dependency information for faker from https://files.pythonhosted.org/packages/cc/9a/74db0cf3115df2f71edcaf8d86ed556195ac31575212c20425820f81bfd0/Faker-20.1.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
    --------------------------------------- 0.0/1.7 MB 660.6 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/1.7 MB 1.3 MB/s eta 0:00:02
   ---- ----------------------------------- 0.2/1.7 MB 1.6 MB/s eta 0:00:01
   -------- ------------------------------- 0.4/1.7 MB 2.3 MB/s eta 0:00:01
   ----------- ---------------------------- 0.5/1.7 MB 2.3 MB/s eta 0:00:01
   ------------------- -------------------- 0.8/1.7 MB 3.3 MB/s eta 0:00:01
   ---------------------- ----------------- 1.0/1.7 MB 3.0 MB/s eta 0:00:01
   ----------------------------- ---------- 1.3/1.7 MB 3.7 MB/s eta 0:00:01
   ------------------------------------ --- 1.6/1.7 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------  1.7/1

In [3]:
import psycopg2
from faker import Faker
import json
from datetime import datetime, timedelta
import random

# Generate Data Dummy

In [10]:
# Buat yang information biar jenis transaksi +  kode

def generate_transaction_code(transaction_type):
    letters = transaction_type.split()[0][:1] + transaction_type.split()[0][2:3]
    random_numbers = ''.join([str(random.randint(0, 9)) for _ in range(7)])
    return f'{letters} - {random_numbers}'


# Function Generate data

def generate_dummy_data(num_records):
    fake = Faker()
    data = []

    for _ in range(num_records):
        trx_date = datetime.now() - timedelta(days=random.randint(1, 365))
        trx_time = f"{random.randint(0, 23):02d}:{random.randint(0, 59):02d}:{random.randint(0, 59):02d}"

        transaction_type = fake.random_element(elements=('WITHDRAWAL TRANSFER (ATM)', 'GOPAY',
                                                         'DEPOSIT TRANSFER (ATM)', 'DEPOSIT CROSS BORDER',
                                                         'WITHDRAWAL TRANSFER', 'REVERSAL', 'BIAYA ADMIN',
                                                         'PAYMENT CC', 'PAYMENT TELCO PASCABAYAR',
                                                         'ONLINE SHOPPING', 'INVESTMENT', 'LOAN REPAYMENT',
                                                         'WITHDRAWAL CASH', 'TRANSFER BETWEEN ACCOUNTS',
                                                         'UTILITY BILL PAYMENT', 'RENT PAYMENT',
                                                         'SALES REFUND', 'LOYALTY POINTS REDEMPTION'))

        transaction_code = generate_transaction_code(transaction_type)

        record = {
            "ACCOUNT_NUMBER": fake.random_number(digits=random.randint(5, 14)),
            "TRX_TYPE": fake.random_element(elements=('Ewallet', 'Bank Transfer',
                                                      'Credit Card', 'Virtual Account', 'Cash',
                                                      'Cryptocurrency', 'PayPal', 'Mobile Payment',
                                                      'Cheque', 'Money Order', 'Game', 'Direct Deposit',
                                                      'Wire Transfer', 'Prepaid Card', 'Google Pay',
                                                      'Apple Pay', 'Venmo', 'Zelle', 'Western Union',
                                                      'Alipay', 'WeChat Pay')),

            "AMOUNT": fake.random_int(min=-50000000, max=50000000), # -50 juta sampe 50 juta

            "FLAG_DB_CR": fake.random_element(elements=('D', 'C')),  # D = Debit dan C = Credit

            "INFORMATION": f'{transaction_type} {transaction_code}',

            "TRX_DATE": int(trx_date.timestamp()) * 1000,  # Convert to milliseconds
            "REFERENCE_NUMBER": fake.random_number(digits=6),
            "CURRENCY": 'IDR',
            "TRX_TIME": trx_time,
            "CIF": fake.random_number(digits=10),
            "FITUR": fake.random_element(elements=('Banking Services','Gopay', 'OVO', 'Dana'))
        }
        data.append(record)
    return data

In [6]:
# Jumlah data dummy yang diinginkan
num_records = 10
dummy_data = generate_dummy_data(num_records)
dummy_data

[{'ACCOUNT_NUMBER': 63196857811,
  'TRX_TYPE': 'Ewallet',
  'AMOUNT': -8464904,
  'FLAG_DB_CR': 'C',
  'INFORMATION': 'UTILITY BILL PAYMENT UI - 6763153',
  'TRX_DATE': 1678545009000,
  'REFERENCE_NUMBER': 745133,
  'CURRENCY': 'IDR',
  'TRX_TIME': '10:17:37',
  'CIF': 6923982550,
  'FITUR': 'Dana'},
 {'ACCOUNT_NUMBER': 7244047852700,
  'TRX_TYPE': 'Western Union',
  'AMOUNT': 29189435,
  'FLAG_DB_CR': 'C',
  'INFORMATION': 'REVERSAL RV - 2993477',
  'TRX_DATE': 1691850609000,
  'REFERENCE_NUMBER': 581321,
  'CURRENCY': 'IDR',
  'TRX_TIME': '03:24:27',
  'CIF': 9181603403,
  'FITUR': 'Banking Services'},
 {'ACCOUNT_NUMBER': 3804289168,
  'TRX_TYPE': 'Zelle',
  'AMOUNT': -30557616,
  'FLAG_DB_CR': 'C',
  'INFORMATION': 'REVERSAL RV - 2261188',
  'TRX_DATE': 1687357809000,
  'REFERENCE_NUMBER': 15675,
  'CURRENCY': 'IDR',
  'TRX_TIME': '14:47:08',
  'CIF': 6606101370,
  'FITUR': 'Gopay'},
 {'ACCOUNT_NUMBER': 31079527768461,
  'TRX_TYPE': 'Money Order',
  'AMOUNT': 30597922,
  'FLAG_DB_CR

# Simpan PostgreSQL

In [12]:
def insert_data_to_postgresql(data):
    connection = psycopg2.connect(
        host="localhost",
        database="postgres2",
        user="postgres",
        password="1201"
    )

    cursor = connection.cursor()

    for record in data:
        insert_query = """
            INSERT INTO user_transaction (
                ACCOUNT_NUMBER, TRX_TYPE, AMOUNT, FLAG_DB_CR,
                INFORMATION, TRX_DATE, REFERENCE_NUMBER, CURRENCY,
                TRX_TIME, CIF, FITUR
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (
            record["ACCOUNT_NUMBER"], record["TRX_TYPE"], record["AMOUNT"],
            record["FLAG_DB_CR"], record["INFORMATION"], record["TRX_DATE"],
            record["REFERENCE_NUMBER"], record["CURRENCY"], record["TRX_TIME"],
            record["CIF"], record["FITUR"]
        ))

    connection.commit()
    connection.close()

In [13]:
# Jumlah data dummy yang diinginkan

num_records = 150000 # karena takut laptop tidak kuat kalau 500k
dummy_data = generate_dummy_data(num_records)
insert_data_to_postgresql(dummy_data)